In [1]:
import pandas as pd
import numpy as np
from Class_Network import *
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn import datasets
from sklearn.decomposition import PCA
import copy

# import some data to play with
#iris.data shape of (150,4)
iris = datasets.load_iris()
X = iris.data[:, :2]  # we only take the first two features.
y = iris.target

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
data = datasets.load_iris()
y = pd.get_dummies(data.target).values
X = data.data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 80)


In [3]:
nnn = 3 # number of splits selected...
kf = KFold(nnn)
kf.get_n_splits(X_train)
kf.split(X_train)
Xtr = []
Xtest = []
ytr = []
ytest = []
for train_index, test_index in kf.split(X_train):
    #train
    #print("TRAIN:", train_index, "TEST:", test_index)
    Xtr.append(X_train[train_index])
    ytr.append(y_train[train_index])
    #test
    Xtest.append(X_train[test_index])
    ytest.append(y_train[test_index])
n_input = np.shape(X_train)[1]
n_output = np.shape(y_test)[1]

In [4]:
acc_vec = np.zeros((2,len(Xtest)))
tr_attempt = [0, 0.2, 0.4, 0.5, 0.6, 0.8, 1] # threshold attempt
acc_global = np.zeros((2,len(tr_attempt))) # accuracy

for j in range(len(tr_attempt)): # looping over thresholds
    for i in range(len(Xtest)): # looping over training folds
        net = Network([n_input, 3, n_output], type='classifier', prior_dist_weight = ['gaussian','gaussian'], prior_dist_bias = ['gaussian', 'gaussian','gaussian'])
        net.Learn(Xtr[i], ytr[i], epochs=200, lrate=1, times = 1000, threshold = tr_attempt[j], bootstrap = False, progress=True)
        CE = net.Evaluate(Xtr[i], ytr[i])
        accuracy = net.ClassificationAccuracy(Xtr[i], ytr[i])
        acc_vec[0][i] = accuracy
        CE = net.Evaluate(Xtest[i], ytest[i])
        accuracy = net.ClassificationAccuracy(Xtest[i], ytest[i])
        acc_vec[1][i] = accuracy
    #print(acc_vec[0].sum() / len(Xtest))
    #print(j)
    acc_global[0][j] = acc_vec[0].sum() / len(Xtest)
    acc_global[1][j] = acc_vec[1].sum() / len(Xtest)
    
plt.figure(0)
plt.plot(tr_attempt,acc_global[0])
plt.title("Training accuracy vs. threshold") #training folds accuracy

plt.figure(1)
plt.plot(tr_attempt,acc_global[1])
plt.title("Test accuracy vs. threshold") #test folds accuracy

ValueError: setting an array element with a sequence.

In [ ]:
# Evaluate it before training
#prior dist for weight and bias, is the number of layers..
net = Network([n_input, 3, np.shape(y_train)[1]], type='classifier', prior_dist_weight = ['gaussian','gaussian'], prior_dist_bias = ['gaussian','gaussian'])
#net_boot = copy.deepcopy(net) #same initiliazed weights therefore..
net_boot =  Network([n_input, 3, np.shape(y_train)[1]], type='classifier', prior_dist_weight = ['gaussian','gaussian'], prior_dist_bias = ['gaussian','gaussian'])
net.Learn(X_train.copy(), y_train.copy(), epochs=300, lrate=1, times = 1000, threshold = 0, bootstrap = False, progress=True)
net_boot.Learn(X_train.copy(), y_train.copy(), epochs=300, lrate=1, times = 1000, threshold = 0, bootstrap = True, progress=True)

CE = net.Evaluate(X_train, y_train)
accuracy = net.ClassificationAccuracy(X_train, y_train)
print('Training Set: Gaussian')
print('Cross Entropy = '+str(CE))
print('     Accuracy = '+str(accuracy*100.)+'%')

CE = net_boot.Evaluate(X_train, y_train)
accuracy = net_boot.ClassificationAccuracy(X_train, y_train)
print('Training Set: Bootstrap')
print('Cross Entropy = '+str(CE))
print('     Accuracy = '+str(accuracy*100.)+'%')

print('Test Set: Gaussian')
CE = net.Evaluate(X_test, y_test)
accuracy = net.ClassificationAccuracy(X_test, y_test)
print('Cross Entropy = '+str(CE))
print('     Accuracy = '+str(accuracy*100.)+'%')

print('Test Set: Bootstrap')
CE = net_boot.Evaluate(X_test, y_test)
accuracy = net_boot.ClassificationAccuracy(X_test, y_test)
print('Cross Entropy = '+str(CE))
print('     Accuracy = '+str(accuracy*100.)+'%')

In [ ]:
print(net.weight_matrix[0].sigma)
print(net_boot.weight_matrix[1].sigma)

In [4]:
net = Network([n_input, 3, np.shape(y_train)[1]], type='classifier', prior_dist_weight = ['gaussian','gaussian'], prior_dist_bias = ['gaussian','gaussian'])
net.Learn(X_train, y_train, epochs=200, lrate=1, times = 1000, threshold = 0, bootstrap = True, progress=True)


In [5]:
print(net.weight_matrix[0].sigma)
print(net.weight_matrix[1].sigma)

CE = net.Evaluate(X_train, y_train)
accuracy = net.ClassificationAccuracy(X_train, y_train)
print('Training Set: Bootstrap')
print('Cross Entropy = '+str(CE))
print('     Accuracy = '+str(accuracy*100.)+'%') # 92

print('Test Set: Bootstrap')
CE = net.Evaluate(X_test, y_test)
accuracy = net.ClassificationAccuracy(X_test, y_test)
print('Cross Entropy = '+str(CE))
print('     Accuracy = '+str(accuracy*100.)+'%') #  78

[[1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]]
[[1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]]
Training Set: Bootstrap
Cross Entropy = 0.9633835466961471


TypeError: can't multiply sequence by non-int of type 'float'

In [ ]:
plt.plot(net.cost_history);
#with gaussian train accuary=, test accuracy=)

In [ ]:
plt.plot(net_boot.cost_history);
#with bootstrap

In [ ]:
def OutputHist(net, samples, bootbool, test):
    '''
        Given a network and a test data point will produce histograms of the output 
        at each neuron

        Inputs:
          net: The trained network object.
          samples: The number of samplings of weights and biases used.
          bootbool: Boolean value, whether bootstrap is used or not.
          test: A test data point (a single data point).
          
    '''
    #samples = 1000 # number of times to sample weights and biases
    n = np.shape(net.lyr[-1].h)[1] # number of output nodes
    for i in range(0, np.shape(net.lyr[-1].h)[1]): #loop over output nodes
        y_stack = net.FeedForward(test,bootbool) # first sampling
        for j in range(0, samples-1): # loop over samplings of W.. and b.. to produce output histogram
            y = net.FeedForward(test,bootbool) #data, bootstrap boolean. Output -> shape is (samples, # of nodes)
            y_stack = np.vstack((y_stack,y))

    #Plot output distributions
    for i in range(0, n):
        plt.figure(i,figsize=(8,8))
        plt.hist(y_stack[:,i],bins =50,range=(0,1))
        l,r = plt.xlim((0,1))
        plt.title('%d Output Neuron'%(i+1))

In [ ]:
OutputHist(net, samples=500, bootbool=False, test=X_test[15]) #y_test

In [ ]:
y_test[15]